In [14]:
from astropy import constants as const
from astropy import units as u
import math 
import numpy as np
import matplotlib.pyplot as plt
#Function with the calculation
def mag_per_arcsec2 (dist_mod, apparent_mag, half_light_radius):
    if(half_light_radius > 0):
        mag_per_arcsec_squared = math.pi * math.pow(100, dist_mod / 5) 
        mag_per_arcsec_squared *= 2.3815 * math.pow(10, -11)
        mag_per_arcsec_squared *= apparent_mag / (half_light_radius ** 2)
        return mag_per_arcsec_squared 
    else:
        return 0

In [15]:
#All of the values
labels = ['M 32', 'NGC 205', 'And I', 'And XVII', 'And III', 'And V', 
          'And XX', 'And XXXII', 'NGC 147', 'And XXX', 'And XIV', 
          'And XV', 'And II', 'NGC 185', 'And XXIX', 'And VII', 'IC 10',
          'And XXXI', 'LGS 3', 'And VI', 'And XVI']
labels = np.array(list(labels))
apparent = [ 8.13,  8.1 , 13.16, 16.51, 14.87, 15.14, 17.95, 12.15,  
            9.5, 16.17, 16.  , 15.98, 12.47,  9.2 , 16.02, 11.21,  
            9.5 , 12.7, 14.3 , 12.97, 16.09] #in the V band
apparent = np.array(list(apparent))
distance_mod = [24.53, 24.58, 24.36, 24.31, 24.37, 24.44, 24.35, 24.45, 
                24.15, 24.17, 24.50, 23.98, 24.07, 23.95, 24.32, 24.41, 
                24.5, 24.4, 24.43, 24.47, 23.39]
distance_mod = np.array(list(distance_mod))
half_light_radii = [0, 0, 638, 265, 403, 336, 235, 252, 0, 267, 406,
                    270, 1126, 0, 0, 739, 0, 0, 0, 454, 136]
half_light_radii = np.array(list(half_light_radii))

In [16]:
print(len(half_light_radii) == len(apparent))
print(len(distance_mod) == len(apparent))

True
True


In [17]:
mag_per_square_arcsec = np.empty(len(half_light_radii))
for index in range(len(half_light_radii)):
    mag_per_square_arcsec[index] = mag_per_arcsec2(distance_mod[index], 
                apparent[index], half_light_radii[index])
usable_objects = np.nonzero(mag_per_square_arcsec)
print(mag_per_square_arcsec[usable_objects])
old_labels = labels

[1.34157510e-05 9.31656426e-05 3.83443460e-05 5.99028905e-05
 1.33637760e-04 8.62532871e-05 7.90110723e-05 4.58214021e-05
 6.40987714e-05 3.12456908e-06 8.91902116e-06 2.88954012e-05
 1.47733398e-04]


In [18]:
usable_values = (- 1) * np.log10(mag_per_square_arcsec[usable_objects])
labels = old_labels[usable_objects]
fig, ax = plt.subplots(figsize=(20, 5))
indices = np.arange(len(usable_values))
fig.dpi = 1000
bottom = int(np.amin(usable_values))
plt.bar(x = indices, height = usable_values - bottom, 
        bottom = bottom )
plt.xticks(indices, labels)
plt.yticks(np.arange(int(np.amin(usable_values)), int(np.amax(usable_values)) + 1,  
                     step = .1 * (int(np.amax(usable_values)) -  int(np.amin(usable_values)) + 1) ))
plt.grid(which='major', color='black')
plt.grid(which='minor', color='grey', alpha=0.4)
plt.minorticks_on()
plt.ylabel("- log10( m / arcsec^2)")
plt.tick_params(axis='x', which='minor', bottom = False)
plt.grid(axis='x', which = 'minor', alpha = 0)
plt.title("Surface Brightness of M31 Satellites")

Text(0.5, 1.0, 'Surface Brightness of M31 Satellites')

In [91]:
def calc_SNR (Robject, t, Rsky, n, RN, dark, magnitudes, band):
    if(magnitudes):
        Rsky = convert_mag_to_flux(Rsky, band).value
        Robject = convert_mag_to_flux(Robject, band).value
        #converts to number of photons per square meter per second
    SNR = Robject * t / math.sqrt(Robject * t + Rsky * t * n + (RN ** 2) 
        * n + dark * t * n)
    return SNR

#now to just to make sure it will compile
calc_SNR(Robject = 1, t = 1, Rsky = 1, n = 1, RN = 1, dark = 1, 
        magnitudes = True, band = "J")

4.746847174746851e-17

In [85]:
'''
References: 
https://web.archive.org/web/20180721162252/https://www.
cfa.harvard.edu/~dfabricant/huchra/ay145/mags.html

https://en.wikipedia.org/wiki/Apparent_magnitude#cite_note-UTmags-19

https://het.as.utexas.edu/HET/Software/Astropy-0.4.2/api/astropy.units.
quantity.Quantity.html
'''
def convert_mag_to_flux(mag, band): 
    if(band == 'J' or band == 'J'):
        lam = (1.220 * u.micron).to("m") #wavelength of each photon
        ref_val = 1570 * math.pow(10, -26) * u.jansky
        #based on flux at magnitude 0 in Jy = 10^-26 W m^-2 Hz^-1
        h = const.h.to('J*s')
        c = const.c.to("m/s")
        single_photon_energy = (h * c / lam)
        #since triple spec goes from 1 micron to 2.4 micron
        lower_lam = (1 * u.micron).to("m")
        upper_lam = (2.4 * u.micron).to("m")
        frequency_range = (c / lower_lam) - (c / upper_lam)
        #calculating spectral range in Hz
        flux = math.pow(100, (-mag / 5)) * ref_val * frequency_range 
        flux /= single_photon_energy
        flux = flux.si
        #really it's the number of photons per square meter per second
        return flux
    else:
        print("Band not yet computable")
        return 
    
#now to test that it works
Rsky = 19.2 #magnitudes
Rsky = convert_mag_to_flux(Rsky, "J")
Rsky

<Quantity 3.52306169e-24 1 / (m2 s)>

In [86]:
'''
Now we will use all of the relevant constants for the TSPEC at Palomar
'''
int_time_array = np.arange(1, 301) #integration time in seconds
RN = 10 #electrons
dark = .085 #electrons
Rsky = 19.2 #magnitudes
n = 1024 * 2048 #square pixels